In [63]:
from dotenv import load_dotenv
load_dotenv("api.env")

import pandas as pd
import numpy as np

In [64]:
df = pd.read_csv('data/ground_truth_df.csv')
prompts = [
    f""""{narrative}" Is the person wearing a helmet? Answer 2 if cannot infer from narrative. Answer 1 if yes, and 0 if no. Only output the number."""
    for narrative in df['narrative_1'].tolist()
]

In [65]:
import time
from together import Together
from tqdm import tqdm

client = Together()

INITIAL_BACKOFF = 1  # in seconds

def send_with_retry(prompt):
    INITIAL_BACKOFF = 0.5  # Define backoff constant
    attempt = 0  # Initialize attempt counter
    
    while True:
        try:
            response = client.chat.completions.create(
                model="deepseek-ai/DeepSeek-V3",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=5,
            )
            return response
        except Exception as e:
            backoff = INITIAL_BACKOFF * (2 ** attempt)
            print(f"Error: {e} (attempt {attempt+1}), retrying in {backoff:.1f}s")
            time.sleep(backoff)
            attempt += 1  # Increment attempt counter
            
            # Add maximum retry condition
            if attempt >= 5:  # Max 5 retries
                print(f"Failed to get response after {attempt} attempts for: {prompt}")
                return None

# Example usage
responses = [send_with_retry(p) for p in tqdm(prompts)]

100%|██████████| 400/400 [20:47<00:00,  3.12s/it]


In [66]:
numbers = np.array([int(response.choices[0].message.content) for response in responses])
(df['helmet_status_human'].to_numpy() == numbers).mean()

0.9875

In [82]:
mismatched_indices = np.where(df['helmet_status_human'] != numbers)[0]
mismatched_df = pd.DataFrame({
    'Index': mismatched_indices,
    'Human Label': df.iloc[mismatched_indices]['helmet_status_human'].values,
    'Model Prediction': numbers[mismatched_indices],
    'Narrative': df.iloc[mismatched_indices]['narrative_1'].values
})

In [85]:
# Use pandas built-in to_latex function for a simple table
latex_table = mismatched_df.to_latex(
    index=False,
    longtable=False,  # Changed to False to prevent multi-page tables
    caption="Mismatched predictions between human labels and model predictions.",
    label="tab:mismatches",
    escape=True,
    column_format="cccp{8cm}"
)

# Save to file
with open('mismatches_table.tex', 'w') as f:
    f.write(latex_table)